# PRÁCTICA 1 - APRENDIZAJE SUPERVISADO - SUPERVISED


## 0. Indice

1. [Importación de Librerías, Lectura y Configuración](#21-importación-de-librerías-lectura-y-configuración)
2. [Metodos Clásicos](#22-metodos-classicos)
    1. [Regresión Lineal](#221-regresion-lineal)
    2. [Regresión Polinomial](#222-regresion-polinomial)
    3. [Regresión Logística](#223-regresion-logistica)


## 2.1. Importación de Librerías, Lectura y Configuración


Es este apartado incluiremos las configuraciones iniciales pertinentes para la realizacion del aprendizaje supervisado.


### 2.1.1. Imports


Procederemos ha realizar los imports pertinentes


In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, r2_score, f1_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

### 2.1.2. Lectura de Dataframes


-   En este apartado procederemos a la lectura del dataframe creado en el notebook de preprocesado.
-   Tendremos los siguientes datasets:
    -   `X_train`: dataset con la información de los clientes destinado al training.
    -   `X_test`: dataset con la infromación de los clientes destinado a la prueba del rendimiento del modelo.
    -   `y_train`: dataset con la información de la variable objetivo destinado al training.
    -   `y_test`: dataset con la información de la variable objetivo destinado a la prueba del rendimiento del modelo.


In [3]:
X_train = pd.read_csv("./data/output/clean_train_data.csv")
X_test = pd.read_csv("./data/output/clean_test_data.csv")
y_train = pd.read_csv("./data/output/clean_train_label.csv")
y_test = pd.read_csv("./data/output/clean_test_label.csv")

## 2.2. Metodos Classicos


Se realizaran pruebas con los metodos:

1. **Regresión Lineal**
2. **Regresión Polinomial**
3. **Regresión Lógica**


### 2.2.1. Regresion Lineal

-   Para realizar la regresion lineal procederemos a utilizar el algoritmo de `RidgeClassifier`.
-   Para poder testear el modelo utilizaremos la metrica de `f1_score`


In [7]:
ridge_clf = RidgeClassifier()

# Definimos el rango de valores para cada hiperparámetro
param_grid = {
    'alpha': [0.1, 1, 10, 100],       # Valores de regularización
    'solver': ['auto', 'svd', 'cholesky', 'sag', 'saga'], # Algoritmos para resolver el modelo
    'tol': [1e-3, 1e-4, 1e-5],        # Tolerancias para el criterio de parada
    'class_weight': [None, 'balanced'] # Peso de las clases
}

# Configuramos el GridSearchCV
grid_search = GridSearchCV(estimator=ridge_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

y_train_pred = grid_search.predict(X_train)
f1_train = f1_score(y_train, y_train_pred)
print(f"Train f1 score: {f1_train}")

y_test_pred = grid_search.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)
print(f"Test f1 score: {f1_test}")


C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:1303: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Parameters: {'alpha': 1, 'class_weight': None, 'solver': 'sag', 'tol': 0.001}
Train f1 score: 0.5475098736500549
Test f1 score: 0.5487158706007386


Best Parameters: {'alpha': 1, 'class_weight': None, 'solver': 'sag', 'tol': 0.001}

Train f1 score: 0.5475098736500549

Test f1 score: 0.5487158706007386

-   Como podemos ver con los valores de train y test, **no hay una realacion lineal fuerte** entre las variables por lo que podemos deducir que no hay una alta relacion lineal entre las variables predictorias y la variable objetivo.
-   También podemos ver que la diferencia entre training y testing soy muy pequeñas por lo que parece que **no hay overfitting**.


### 2.2.2. Regresion Polinomial

-   Para realizar la regresion polinomial procederemos a utilizar el algoritmo de `PolinomialFeatures` para añadir el cuadrado del valor de las caracteristicas y posteriormente entrenarlo con una `RidgeClassifier`.
-   Para poder testear el modelo utilizaremos el metodo de `f1_score`


In [ ]:
poly = PolynomialFeatures()


X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

ridge = RidgeClassifier()

# Definimos el grid de parámetros
param_grid = {
    'alpha': [0.1, 1, 10, 100],     # Valores de regularización de Ridge
    'solver': ['auto', 'sag'] # Solvers para el RidgeClassifier
}

# Configuramos el GridSearchCV con validación cruzada
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)

grid_search.fit(X_train_poly, y_train)

print("Best Parameters:", grid_search.best_params_)

y_train_pred = grid_search.predict(X_train)
f1_train = f1_score(y_train, y_train_pred)
print(f"Train f1 score: {f1_train}")

y_test_pred = grid_search.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)
print(f"Test f1 score: {f1_test}")


Best Parameters: {'alpha': 1, 'solver': 'sag'}

Train f1 score: 0.5821903894326542

Test f1 score: 0.5783456567980094


-   Como podemos volver a ver, el modelo polinomial **sigues sin campturar adecuadamente la relacion entre las caracteristicas y la variable objetivo**.
-   También podemos ver que la diferencia entre training y testing soy muy pequeñas por lo que parece que **no hay overfitting**.


### 2.2.3. Regresion Logistica

-   Para realizar la regresion logistica procederemos a utilziar el algoritmo de `LogisticalRegression` de sklearn.
-   Para poder testear el modelo utilizaremos el método de `f1_score`.


In [6]:

from sklearn.linear_model import LogisticRegression


model = LogisticRegression(max_iter=1000)
params = {
    'C': [0.0001, 0.001],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['saga', 'liblinear']
}
f1_scorer = make_scorer(f1_score)

grid_search = GridSearchCV(model, params, scoring=f1_scorer, cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

y_train_pred = grid_search.predict(X_train)
f1_train = f1_score(y_train, y_train_pred)
print(f"Train f1 score: {f1_train}")

y_test_pred = grid_search.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)
print(f"Test f1 score: {f1_test}")

C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-pack

Best Parameters: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
Train f1 score: 0.5624431268951142
Test f1 score: 0.5634917140047264


Mejores hiperparámetros: {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}

Mejor F1-score en validación: 0.5863199556905928


-   Como podemos observar, usando una regresion logistica **tampoco consigue valores esperanzadores** aun intentando seleccionar los mejores hiperparametros, por lo que probaremos con algoritmos mas complejos.


## 2.3. Árboles de decisión


-   Para realizar una Árbol de decisión, en nuestro caso utilizaremos el `DecisionTreeClassifier` en vez del `DecisionTreeRegressor` debido a que nuestra variable objetivo es categórica.
-   Primero entrenaremos un `DecisionTreeClassifier` de manera simple sin ajustar ningun hiperparametro para ver que puntuacion nos da. Posteriormente probaremos de ajustar los hiperparametros para mejorar el modelo.


#### Sin hiperparametros ajutados


In [10]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
f1_train = f1_score(y_train, y_train_pred)
print(f"Train f1 score: {f1_train}")

y_test_pred = dt.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)
print(f"Test f1 score: {f1_test}")

Train f1 score: 1.0
Test f1 score: 0.6782165063506403


Train f1 score: 1.0

Test f1 score: 0.6782165063506403


Podemos observar que sin ajustar hiperparametros solo nos da una puntuación del ~0,69 en los tests y un 1 en el training, una clara señal de **overfitting**, por lo que intentaremos remediar esto ajustando los hiperparametros.

Hiperparametros a ajustar:

-   **max_depth**: limitar la profundidad del árbol.
-   **min_samples_split** y **min_samples_leaf**: numero mínimo de muestras por nodo.
-   **ccp_alpha**: poda del árbol


In [33]:
params = {
    'max_depth': [22],
    'min_samples_split': [2],
    'min_samples_leaf': [3],
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), params, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

mejor_modelo = grid_search.best_estimator_
print("Mejores hiperparámetros:", grid_search.best_params_)

y_train_pred = mejor_modelo.predict(X_train)
f1_train = f1_score(y_train, y_train_pred)  # Para problemas de clasificación
print(f'F1-score en entrenamiento: {f1_train}')

# Evaluar el rendimiento en el conjunto de prueba
y_test_pred = mejor_modelo.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)
print(f'F1-score en prueba: {f1_test}')

Mejores hiperparámetros: {'max_depth': 22, 'min_samples_leaf': 3, 'min_samples_split': 2}
F1-score en entrenamiento: 0.8991506481895396
F1-score en prueba: 0.6771604938271605


Mejores hiperparámetros: {'max_depth': 22, 'min_samples_leaf': 3, 'min_samples_split': 2}

F1-score en entrenamiento: 0.8991506481895396

F1-score en prueba: 0.6771604938271605


## 2.3. KNN


In [36]:
# Probar varios valores de k
valores_k = range(1, 10)

y_train_knn = y_train.values.ravel()
y_test_knn = y_test.values.ravel()

mejor_k = 0

for k in valores_k:
    knn = KNeighborsClassifier(n_neighbors=k)

    knn.fit(X_train, y_train_knn)

    y_train_pred = knn.predict(X_train)
    f1_train = f1_score(y_train, y_train_pred)  # Para problemas de clasificación
    print(f'F1-score en entrenamiento: {f1_train}, k:{k}')

    y_test_pred = knn.predict(X_test)
    f1_test = f1_score(y_test, y_test_pred)
    print(f'F1-score en prueba: {f1_test}, k:{k}')
    

F1-score en entrenamiento: 1.0, k:1
F1-score en prueba: 0.7937051300337564, k:1
F1-score en entrenamiento: 0.9554959785522789, k:2
F1-score en prueba: 0.7411535548100855, k:2
F1-score en entrenamiento: 0.8626586001176372, k:3
F1-score en prueba: 0.7358455367969405, k:3
F1-score en entrenamiento: 0.8461362282992075, k:4
F1-score en prueba: 0.7036710109559167, k:4
F1-score en entrenamiento: 0.807756369394008, k:5
F1-score en prueba: 0.7109353187529083, k:5
F1-score en entrenamiento: 0.7926448272784076, k:6
F1-score en prueba: 0.681581897869738, k:6
F1-score en entrenamiento: 0.7742645709742524, k:7
F1-score en prueba: 0.6893636785880167, k:7
F1-score en entrenamiento: 0.7620343745198322, k:8
F1-score en prueba: 0.6653256223283882, k:8
F1-score en entrenamiento: 0.751914428102589, k:9
F1-score en prueba: 0.6789900393791986, k:9


F1-score en entrenamiento: 1.0, k:1.

F1-score en prueba: 0.7937051300337564, k:1.


## 2.4. SVM


In [ ]:
parametros = {
    'kernel': ['rbf', 'poly', 'sigmoid']
}

grid_search = GridSearchCV(SVC(), parametros, cv=5, scoring='f1')  # Usar 'neg_mean_squared_error' para SVR
grid_search.fit(X_train, y_train)

# Mejor modelo después del ajuste
mejor_modelo = grid_search.best_estimator_
print("Mejores hiperparámetros:", grid_search.best_params_)

y_train_pred = mejor_modelo.predict(X_train)
f1_train = f1_score(y_train, y_train_pred)  # Para problemas de clasificación
print(f'F1-score en entrenamiento: {f1_train}')

# Evaluar el rendimiento en el conjunto de prueba
y_test_pred = mejor_modelo.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)
print(f'F1-score en prueba: {f1_test}')

C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mejores hiperparámetros: {'kernel': 'poly'}
F1-score en entrenamiento: 0.7024357191255661
F1-score en prueba: 0.6643391111476218


## 2.5. Redes Neuronales


In [ ]:
y_train_mlp = y_train.values.ravel()
y_test_mlp = y_test.values.ravel()

# Crear el modelo
model = Sequential()
model.add(Dense(200, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128, callbacks=[early_stopping])

# Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
2231/2231 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.6670 - loss: 0.6958

KeyboardInterrupt: 

## 2.6. Random Forest


In [32]:
y_train_rf = y_train_mlp
y_test_rf = y_test_mlp

rf = RandomForestClassifier()

params = {
    'random_state': [42],
    'n_estimators': [120, 150, 175],
    'max_depth': [15, 20, 22]
}

grid_search = GridSearchCV(rf, params, cv=5, scoring='f1')  # Usar 'neg_mean_squared_error' para SVR
grid_search.fit(X_train, y_train_rf)

# Mejor modelo después del ajuste
mejor_modelo = grid_search.best_estimator_
print("Mejores hiperparámetros:", grid_search.best_params_)

y_train_pred = mejor_modelo.predict(X_train)
f1_train = f1_score(y_train_rf, y_train_pred)  # Para problemas de clasificación
print(f'F1-score en entrenamiento: {f1_train}')

# Evaluar el rendimiento en el conjunto de prueba
y_test_pred = mejor_modelo.predict(X_test)
f1_test = f1_score(y_test_rf, y_test_pred)
print(f'F1-score en prueba: {f1_test}')


Mejores hiperparámetros: {'max_depth': 22, 'n_estimators': 175, 'random_state': 42}
F1-score en entrenamiento: 0.9988812231959724
F1-score en prueba: 0.8114279286879253
